In [2]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'IPAexGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
#データを整理
df23_24 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_23-24シーズン.csv')
df24_25 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_24-25シーズン.csv')
df_box = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】ボックススコア.csv')
df_game = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】試合データ.csv')
df_team = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】チームマスタ.csv')
df_cluster = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/analysis/B1_clusters_2024-25.csv', header=1)


#リーグのみ抽出
#リーグの試合IDを抽出
cupID = [500, 507]
df_gameID_cupID = df_game[df_game['カップID'].isin(cupID)]
df_gameID_cupID = df_gameID_cupID['試合ID'].unique()

cupID24_25 = [507]
df_gameID_cupID24_25 = df_game[df_game['カップID'].isin(cupID24_25)]
df_gameID_cupID24_25 = df_gameID_cupID24_25['試合ID'].unique()

#試合IDからデータを抽出
df23_24 = df23_24[df23_24['試合ID'].isin(df_gameID_cupID)]
df24_25 = df24_25[df24_25['試合ID'].isin(df_gameID_cupID)]
df_game = df_game[df_game['試合ID'].isin(df_gameID_cupID)]


# 試合ID、チームIDごとにデータを整理
df_box_gameid = df_box.drop(columns=['ホームアウェイ','選手ID','背番号','選手名','スターティングフラグ','プレイタイム'])
df_box_gameid_sum = df_box_gameid[df_box_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()


# # 試合ID、チームIDごとにデータを整理
# df_box24_25_gameid = df_box24_25.drop(columns=['ホームアウェイ','選手ID','背番号','選手名','スターティングフラグ','プレイタイム'])
# df_box24_25_gameid_sum = df_box24_25_gameid[df_box24_25_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()

# 試合IDごとに勝敗を割り当て
# 1. 試合IDごとに得点の最大値を取得
max_scores = df_box_gameid_sum.groupby('試合ID')['得点'].transform('max')
# 2. 自チームの得点が最大なら1（勝ち）、そうでなければ0（負け）
df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

# max_scores = df_box24_25_gameid_sum.groupby('試合ID')['得点'].transform('max')
# df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

# 試合データから失点列を作成（より効率的）
df_game_long = pd.concat([
    df_game[['試合ID', 'ホームチームID', 'アウェイ得点']].rename(
        columns={'ホームチームID': 'チームID', 'アウェイ得点': '失点'}
    ),
    df_game[['試合ID', 'アウェイチームID', 'ホーム得点']].rename(
        columns={'アウェイチームID': 'チームID', 'ホーム得点': '失点'}
    )
], ignore_index=True)

# df_box_gameid_sum に失点を結合
df_box_gameid_sum = df_box_gameid_sum.merge(
    df_game_long[['試合ID', 'チームID', '失点']],
    on=['試合ID', 'チームID'],
    how='left'
)

# 安全に成功率を計算するヘルパー
# ...existing code...
def safe_rate(df, num_col, den_col, out_col, fillna=0):
    if num_col not in df.columns or den_col not in df.columns:
        print(f"警告: 列がありません: {num_col} または {den_col}")
        return
    # 数値化（非数値は NaN）
    df[num_col] = pd.to_numeric(df[num_col], errors='coerce')
    df[den_col] = pd.to_numeric(df[den_col], errors='coerce')
    # 0 を NaN にしてゼロ除算を回避
    denom = df[den_col].replace({0: np.nan})
    # 計算（結果を float に強制）
    df[out_col] = (df[num_col] / denom).astype(float)
    # 数値化の再適用（万が一の非数を排除）
    df[out_col] = pd.to_numeric(df[out_col], errors='coerce')
    # 現実的な範囲に収める（負や 1 より大きい値が出た場合にクリップ）
    df[out_col] = df[out_col].clip(lower=0.0, upper=1.0)
    # オプションで NaN を置換
    if fillna is not None:
        df[out_col] = df[out_col].fillna(fillna)

# 成功率列を安全に作成
safe_rate(df_box_gameid_sum, '2P成功', '2P試投', '2P成功率')
safe_rate(df_box_gameid_sum, '3P成功', '3P試投', '3P成功率')
safe_rate(df_box_gameid_sum, 'フリースロー成功', 'フリースロー試投', 'フリースロー成功率')

# safe_rate(df_box24_25_gameid_sum, '2P成功', '2P試投', '2P成功率')
# safe_rate(df_box24_25_gameid_sum, '3P成功', '3P試投', '3P成功率')
# safe_rate(df_box24_25_gameid_sum, 'フリースロー成功', 'フリースロー試投', 'フリースロー成功率')

mask = df_box_gameid_sum['試合ID'].isin(df_gameID_cupID24_25)
df_box_gameid_sum = df_box_gameid_sum.loc[mask].copy()

df_box24_25_gameid_sum = df_box_gameid_sum[df_box_gameid_sum['試合ID'].isin(df_gameID_cupID24_25)]

df_box_alphas = df_box_gameid_sum[df_box_gameid_sum['チームID']== 745]

def get_cluster(df_team, df_box, names):
    ids = df_team.loc[df_team['チーム名'].isin(names), 'チームID']
    return df_box.loc[df_box['チームID'].isin(ids)].copy()

list_cluster0 = df_cluster[df_cluster['Cluster']==0]['Club'].tolist()
list_cluster1 = df_cluster[df_cluster['Cluster']==1]['Club'].tolist()
list_cluster2 = df_cluster[df_cluster['Cluster']==2]['Club'].tolist()

# --- cluster全体 ---
df_cluster0 = get_cluster(df_team, df_box24_25_gameid_sum, list_cluster0)
df_cluster1 = get_cluster(df_team, df_box24_25_gameid_sum, list_cluster1)
df_cluster2 = get_cluster(df_team, df_box24_25_gameid_sum, list_cluster2)

# --- cluster1（越谷以上） ---
df_cluster1_above = get_cluster(df_team, df_box_gameid_sum, {
    'ファイティングイーグルス名古屋','佐賀バルーナーズ','大阪エヴェッサ',
    '広島ドラゴンフライズ','長崎ヴェルカ'
})

In [30]:
df_cluster1

,試合ID,チームID,ピリオド区分,得点,3P成功,3P試投,2P成功,2P試投,ダンク,フリースロー成功,...,ブロックショット,被ブロックショット,ファストブレイクポイント,2Pインサイドポイント,セカンドチャンスポイント,勝敗,失点,2P成功率,3P成功率,フリースロー成功率
2358,502712,712,216,64,10,30,11,26,1,12,...,1,1,11,20,10,0,74.0,0.423077,0.333333,0.857143
2360,502713,712,216,60,9,32,9,24,0,15,...,3,5,11,18,5,0,75.0,0.375000,0.281250,0.789474
2363,502714,721,198,53,7,19,12,37,0,8,...,1,5,5,20,10,0,82.0,0.324324,0.368421,0.888889
2365,502715,721,198,71,11,32,16,31,2,6,...,1,2,16,28,7,0,79.0,0.516129,0.343750,1.000000
2371,502718,745,216,57,4,23,17,37,1,11,...,6,6,7,30,8,0,75.0,0.459459,0.173913,0.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783,503425,2488,216,85,10,24,20,39,1,15,...,4,4,11,38,18,1,76.0,0.512821,0.416667,0.714286
3785,503426,721,198,91,16,43,18,32,2,7,...,1,4,16,36,10,0,94.0,0.562500,0.372093,1.000000
3787,503427,721,198,88,12,40,20,40,1,12,...,2,1,10,36,18,1,69.0,0.500000,0.300000,0.923077
3789,503428,1638,216,71,10,40,16,26,2,9,...,3,0,9,26,11,0,87.0,0.615385,0.250000,0.600000


In [13]:
#目的変数を'勝敗'としたVIFを計算
df = df_box_gameid_sum.copy()

exp_val = [
    'ブロックショット','ファウル','被ブロックショット','ファストブレイクポイント',
    'アシスト','オフェンスリバウンド','ディフェンスリバウンド','スティール',
    'ターンオーバ','被ファウル','フリースロー成功率','2P成功率','3P成功率','フリースロー試投','2P試投','3P試投'
]

# 列選択（リストで）
x = df[exp_val].copy()

y = df["勝敗"]
x.corr()

vif = np.diag(np.linalg.inv(x.corr().values))
df_vif = pd.DataFrame(vif)
df_vif.columns = ["VIF"]
df_vif.index = x.corr().columns
df_vif.sort_values("VIF")

df_vif_sorted = df_vif.sort_values("VIF")

# CSV エクスポート
df_vif_sorted.to_csv('vif_results.csv', encoding='utf-8-sig')
display(df_vif_sorted)


,VIF
フリースロー成功率,1.012507
ブロックショット,1.059580
ファウル,1.171541
ディフェンスリバウンド,1.184711
被ブロックショット,1.228915
スティール,1.284716
3P成功率,1.335937
ターンオーバ,1.420600
ファストブレイクポイント,1.491873
2P成功率,1.562424


In [5]:
# ===== 標準化したデータ分析 =====
# 全体チームを分析
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_box_gameid_sum[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_box_gameid_sum.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_box_gameid_sum["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display(result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.307109056951627
            Iterations: 116
            Function evaluations: 116
            Gradient evaluations: 116


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                 1438
Model:                          Logit   Df Residuals:                     1421
Method:                           MLE   Df Model:                           16
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.5690
Time:                        14:27:07   Log-Likelihood:                -429.55
converged:                       True   LL-Null:                       -996.75
Covariance Type:            nonrobust   LLR p-value:                1.778e-231
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0554      0.086      0.645      0.519      -0.113       0.224
ブロックショット         0.1260      0.087      1.443      0.149      -0.045       0.297
ファウル            -0.7190      0.100     -7.195      0.000      -0.915      -0.523
被ブロックショット        0.1697      0.093      1.819      0.069      -0.013       0.353
ファストブレイクポイント    -0.1944      0.104     -1.872      0.061      -0.398       0.009
アシスト             0.0623      0.114      0.545      0.586      -0.162       0.286
オフェンスリバウンド       1.0278      0.129      7.980      0.000       0.775       1.280
ディフェンスリバウンド      1.6872      0.122     13.822      0.000       1.448       1.926
スティール            1.1828      0.115     10.300      0.000       0.958       1.408
ターンオーバ          -1.1241      0.112    -10.072      0.000      -1.343      -0.905
被ファウル            0.7679      0.151      5.091      0.000       0.472       1.064
フリースロー成功率        0.4002      0.089      4.493      0.000       0.226       0.575
2P成功率            1.4404      0.124     11.630      0.000       1.198       1.683
3P成功率            1.6542      0.125     13.256      0.000       1.410       1.899
フリースロー試投        -0.0338      0.157     -0.215      0.830      -0.342       0.275
2P試投            -0.6757      0.163     -4.143      0.000      -0.995      -0.356
3P試投            -0.7498      0.141     -5.333      0.000      -1.025      -0.474
================================================================================
"""

In [29]:
# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display(result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values,
})

# 7) オッズ比を計算
coef_df['オッズ比'] = np.exp(coef_df['標準偏回帰係数'])

# 8) 結果を表示
display(coef_df)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.33998499525863146
            Iterations: 64
            Function evaluations: 64
            Gradient evaluations: 64


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                   60
Model:                          Logit   Df Residuals:                       48
Method:                           MLE   Df Model:                           11
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.6681
Time:                        21:38:12   Log-Likelihood:                -12.434
converged:                       True   LL-Null:                       -37.460
Covariance Type:            nonrobust   LLR p-value:                 6.125e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.1068      0.443     -2.499      0.012      -1.975      -0.239
ブロックショット              0        nan        nan        nan         nan         nan
ファウル            -0.0544      0.474     -0.115      0.909      -0.984       0.875
被ブロックショット       -0.1649      0.541     -0.305      0.761      -1.226       0.896
ファストブレイクポイント          0        nan        nan        nan         nan         nan
アシスト                  0        nan        nan        nan         nan         nan
オフェンスリバウンド       1.1257      0.611      1.842      0.065      -0.072       2.323
ディフェンスリバウンド      1.1814      0.514      2.297      0.022       0.173       2.190
スティール            0.5761      0.503      1.146      0.252      -0.409       1.562
ターンオーバ          -0.6558      0.484     -1.354      0.176      -1.605       0.293
被ファウル            1.0017      0.501      2.000      0.046       0.020       1.984
フリースロー成功率        0.1144      0.415      0.276      0.783      -0.698       0.927
2P成功率            0.4981      0.481      1.036      0.300      -0.444       1.440
3P成功率            1.1173      0.571      1.957      0.050      -0.001       2.236
フリースロー試投              0        nan        nan        nan         nan         nan
2P試投            -0.3690      0.502     -0.735      0.463      -1.354       0.616
3P試投                  0        nan        nan        nan         nan         nan
================================================================================
"""

,変数,標準偏回帰係数,オッズ比
0,const,-1.106797,0.330616
1,ブロックショット,0.000000,1.000000
2,ファウル,-0.054409,0.947045
3,被ブロックショット,-0.164914,0.847967
4,ファストブレイクポイント,0.000000,1.000000
5,アシスト,0.000000,1.000000
6,オフェンスリバウンド,1.125711,3.082407
7,ディフェンスリバウンド,1.181356,3.258791
8,スティール,0.576087,1.779064
9,ターンオーバ,-0.655763,0.519046


In [7]:
# # ===== 標準化したデータ分析 =====
# # 1) 標準化（DataFrameで列名保持）
# scaler = StandardScaler()
# X_scaled_arr = scaler.fit_transform(df_cluster1[exp_val])
# X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_cluster1.index)

# # 2) 定数項を追加（DataFrameのまま）
# X_scaled = sm.add_constant(X_scaled, has_constant='add')

# # 3) 目的変数
# y = df_cluster1["勝敗"]

# # 4) L1正則化付きロジスティック回帰
# model = sm.Logit(y, X_scaled)
# result = model.fit_regularized(
#     method='l1',
#     alpha=1,
#     maxiter=1000
# )

# # 5) 結果表示
# display(result.summary())

# # 6) 標準偏回帰係数を抽出（列名維持）
# coef_df = pd.DataFrame({
#     '変数': result.params.index,
#     '標準偏回帰係数': result.params.values
# })

In [ ]:
# ===== 標準化したデータ分析 =====
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_cluster1_above[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_cluster1_above.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_cluster1_above["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display('クラスター1上位',result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3352866661817437
            Iterations: 81
            Function evaluations: 81
            Gradient evaluations: 81


'クラスター1上位'

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                  300
Model:                          Logit   Df Residuals:                      284
Method:                           MLE   Df Model:                           15
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.5623
Time:                        14:27:07   Log-Likelihood:                -90.351
converged:                       True   LL-Null:                       -206.44
Covariance Type:            nonrobust   LLR p-value:                 5.705e-41
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.4133      0.185     -2.234      0.025      -0.776      -0.051
ブロックショット         0.2984      0.194      1.540      0.124      -0.081       0.678
ファウル            -0.7949      0.215     -3.691      0.000      -1.217      -0.373
被ブロックショット       -0.1023      0.193     -0.531      0.595      -0.480       0.275
ファストブレイクポイント    -0.1907      0.212     -0.901      0.367      -0.605       0.224
アシスト            -0.0707      0.229     -0.309      0.757      -0.519       0.378
オフェンスリバウンド       0.6147      0.200      3.066      0.002       0.222       1.008
ディフェンスリバウンド      1.4622      0.237      6.176      0.000       0.998       1.926
スティール            0.9912      0.226      4.381      0.000       0.548       1.435
ターンオーバ          -0.6175      0.197     -3.140      0.002      -1.003      -0.232
被ファウル            0.5348      0.328      1.631      0.103      -0.108       1.177
フリースロー成功率        0.4884      0.185      2.634      0.008       0.125       0.852
2P成功率            1.4285      0.253      5.640      0.000       0.932       1.925
3P成功率            1.5263      0.255      5.996      0.000       1.027       2.025
フリースロー試投         0.3541      0.332      1.068      0.286      -0.296       1.004
2P試投                  0        nan        nan        nan         nan         nan
3P試投            -0.3471      0.199     -1.743      0.081      -0.738       0.043
================================================================================
"""

In [41]:
# ===== 標準化したデータ分析 =====
# クラスタ0
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_cluster0[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_cluster0.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_cluster0["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display('クラスタ0',result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df_cluster0 = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values,
})

# 7) オッズ比を計算
coef_df_cluster0['オッズ比'] = np.exp(coef_df_cluster0['標準偏回帰係数'])

# 8) 結果を表示
display(coef_df_cluster0)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3089762614284867
            Iterations: 99
            Function evaluations: 99
            Gradient evaluations: 99


'クラスタ0'

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                  659
Model:                          Logit   Df Residuals:                      642
Method:                           MLE   Df Model:                           16
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.5442
Time:                        21:44:58   Log-Likelihood:                -190.98
converged:                       True   LL-Null:                       -419.00
Covariance Type:            nonrobust   LLR p-value:                 6.138e-87
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.5229      0.156      9.752      0.000       1.217       1.829
ブロックショット         0.0670      0.130      0.515      0.607      -0.188       0.322
ファウル            -0.6015      0.135     -4.443      0.000      -0.867      -0.336
被ブロックショット        0.3523      0.139      2.531      0.011       0.079       0.625
ファストブレイクポイント    -0.2619      0.160     -1.636      0.102      -0.576       0.052
アシスト             0.1426      0.180      0.792      0.428      -0.210       0.496
オフェンスリバウンド       0.7156      0.194      3.688      0.000       0.335       1.096
ディフェンスリバウンド      1.6445      0.178      9.257      0.000       1.296       1.993
スティール            1.2136      0.175      6.927      0.000       0.870       1.557
ターンオーバ          -0.8257      0.153     -5.386      0.000      -1.126      -0.525
被ファウル            0.6171      0.221      2.796      0.005       0.185       1.050
フリースロー成功率        0.3954      0.130      3.037      0.002       0.140       0.651
2P成功率            1.4807      0.188      7.872      0.000       1.112       1.849
3P成功率            1.6157      0.193      8.358      0.000       1.237       1.995
フリースロー試投        -0.1323      0.236     -0.560      0.575      -0.595       0.330
2P試投            -0.3643      0.239     -1.524      0.128      -0.833       0.104
3P試投            -0.6854      0.212     -3.229      0.001      -1.101      -0.269
================================================================================
"""

,変数,標準偏回帰係数,オッズ比
0,const,1.522923,4.585610
1,ブロックショット,0.067001,1.069297
2,ファウル,-0.601464,0.548009
3,被ブロックショット,0.352283,1.422310
4,ファストブレイクポイント,-0.261947,0.769551
5,アシスト,0.142614,1.153284
6,オフェンスリバウンド,0.715633,2.045482
7,ディフェンスリバウンド,1.644504,5.178442
8,スティール,1.213602,3.365587
9,ターンオーバ,-0.825689,0.437933


In [42]:
# ===== 標準化したデータ分析 =====
# 越谷アルファーズ
# 1) 標準化（DataFrameで列名保持）

df_alphas = df_cluster1[df_cluster1['チームID']==745]
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_alphas[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_alphas.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_alphas["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display('クラスタ0',result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df_alphas = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values,
})

# 7) オッズ比を計算
coef_df_alphas['オッズ比'] = np.exp(coef_df_alphas['標準偏回帰係数'])

# 8) 結果を表示
display(coef_df_alphas)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.33998499525863146
            Iterations: 64
            Function evaluations: 64
            Gradient evaluations: 64


'クラスタ0'

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                   60
Model:                          Logit   Df Residuals:                       48
Method:                           MLE   Df Model:                           11
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.6681
Time:                        21:45:04   Log-Likelihood:                -12.434
converged:                       True   LL-Null:                       -37.460
Covariance Type:            nonrobust   LLR p-value:                 6.125e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.1068      0.443     -2.499      0.012      -1.975      -0.239
ブロックショット              0        nan        nan        nan         nan         nan
ファウル            -0.0544      0.474     -0.115      0.909      -0.984       0.875
被ブロックショット       -0.1649      0.541     -0.305      0.761      -1.226       0.896
ファストブレイクポイント          0        nan        nan        nan         nan         nan
アシスト                  0        nan        nan        nan         nan         nan
オフェンスリバウンド       1.1257      0.611      1.842      0.065      -0.072       2.323
ディフェンスリバウンド      1.1814      0.514      2.297      0.022       0.173       2.190
スティール            0.5761      0.503      1.146      0.252      -0.409       1.562
ターンオーバ          -0.6558      0.484     -1.354      0.176      -1.605       0.293
被ファウル            1.0017      0.501      2.000      0.046       0.020       1.984
フリースロー成功率        0.1144      0.415      0.276      0.783      -0.698       0.927
2P成功率            0.4981      0.481      1.036      0.300      -0.444       1.440
3P成功率            1.1173      0.571      1.957      0.050      -0.001       2.236
フリースロー試投              0        nan        nan        nan         nan         nan
2P試投            -0.3690      0.502     -0.735      0.463      -1.354       0.616
3P試投                  0        nan        nan        nan         nan         nan
================================================================================
"""

,変数,標準偏回帰係数,オッズ比
0,const,-1.106797,0.330616
1,ブロックショット,0.000000,1.000000
2,ファウル,-0.054409,0.947045
3,被ブロックショット,-0.164914,0.847967
4,ファストブレイクポイント,0.000000,1.000000
5,アシスト,0.000000,1.000000
6,オフェンスリバウンド,1.125711,3.082407
7,ディフェンスリバウンド,1.181356,3.258791
8,スティール,0.576087,1.779064
9,ターンオーバ,-0.655763,0.519046


In [47]:
coef_all = coef_df_alphas.merge(coef_df_cluster0, on='変数')
coef_all

# 数値列を小数点3桁に丸める
numeric_cols = coef_all.select_dtypes(include=[np.number]).columns
coef_all[numeric_cols] = coef_all[numeric_cols].round(3)

# 結果表示
display(coef_all)

# CSV出力
coef_all.to_csv('coef_all_comparison.csv', index=False, encoding='utf-8-sig')

,変数,標準偏回帰係数_x,オッズ比_x,標準偏回帰係数_y,オッズ比_y
0,const,-1.107,0.331,1.523,4.586
1,ブロックショット,0.000,1.000,0.067,1.069
2,ファウル,-0.054,0.947,-0.601,0.548
3,被ブロックショット,-0.165,0.848,0.352,1.422
4,ファストブレイクポイント,0.000,1.000,-0.262,0.770
5,アシスト,0.000,1.000,0.143,1.153
6,オフェンスリバウンド,1.126,3.082,0.716,2.045
7,ディフェンスリバウンド,1.181,3.259,1.645,5.178
8,スティール,0.576,1.779,1.214,3.366
9,ターンオーバ,-0.656,0.519,-0.826,0.438
